# Predict median house values in Californian districts

In [19]:
#1) Use a Random Forest Regressor as your estimator and perform 3-fold Grid Search cross-validation with
# parameters: n estimators: [5, 15, 25], and max features: [2, 4, 8]. 


# import  packages
import os
import tarfile
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [8]:
# set the url and the file name
housing_url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
housing_tgz = "housing.tgz"

# download the data
urllib.request.urlretrieve(housing_url, housing_tgz)

# unzip the data: this creates a file <housing.csv> in the folder of this notebook
housing_tgz = tarfile.open(housing_tgz)
housing_tgz.extractall()
housing_tgz.close()

In [9]:
# read the data using pandas
housing = pd.read_csv("housing.csv")

In [10]:
# display the data
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [11]:
# create income_cat as a new feature based on the median_income
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [12]:
# use stratified split to create the training and test sets
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [13]:
# create the target value of this problem and remove it from the input features 
housing_labels = strat_train_set['median_house_value'].copy()
housing = strat_train_set.drop(['median_house_value', 'income_cat'], axis=1) 

In [14]:
# we need to dummify the categorical variable  
housing_cat = housing[["ocean_proximity"]].copy()
housing_num = housing.drop("ocean_proximity", axis=1)

In [15]:
# use a custom class to create some extra features
from sklearn.base import BaseEstimator, TransformerMixin

# # column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [20]:
# prepare the numeric and the categorical features
num_features = list(housing_num)
cat_features = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler())
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", OneHotEncoder(), cat_features)
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [21]:
# set the parameters grid
param_grid = [{'n_estimators': [5, 15, 25], 'max_features': [2, 4, 8]}]

In [23]:
# create a Random Forest Regressor model and use GridSearchCV to apply 3-fold cv
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor()
grid_search = GridSearchCV(rf, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

In [24]:
# fit the grid to find the best parameters
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 8],
                          'n_estimators': [5, 15, 25]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
# get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
print (best_params)

{'max_features': 8, 'n_estimators': 25}


## comment:
We observe that both the maximum number of features and the number of estimators of the best model take values equal to the upper limit of the values considered during the grid search. This result suggests that it would be beneficial to  run again  the calculation increasing the the upper limits of hyperparameters in order to potentially achieve better results. Therefore, we can conclude that the current results are not good enough.

In [27]:
# 2) What are the three most important features of your best model?

# print the feature importance 
extra_features = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]

feature_importances = grid_search.best_estimator_.feature_importances_
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_features = list(cat_encoder.categories_[0])
features = num_features + extra_features + cat_one_hot_features
for importance, feature in sorted(zip(feature_importances, features), reverse=True):
    print (f"{feature:20s} {importance}")

median_income        0.3481024071877043
INLAND               0.144573572050658
pop_per_hhold        0.115516826058433
longitude            0.07400091920099429
bedrooms_per_room    0.06824952950438436
rooms_per_hhold      0.06315478006077326
latitude             0.06299724596872852
housing_median_age   0.0428743134366594
total_rooms          0.016137654152075312
total_bedrooms       0.014359128381574732
households           0.014289465340913778
population           0.01427451011720191
<1H OCEAN            0.014194592311435668
NEAR OCEAN           0.0036279407324441336
NEAR BAY             0.003545765609285442
ISLAND               0.00010134988673395523


## comment:
According to the above results we conclude that the most important features are median_income, INLAND and pop_per_hhold